In [ ]:
#| default_exp learning_interface

# Learning Interface

> Gradio interface

## Setup

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Tuple, Any
import gradio as gr
import logging
from pathlib import Path
import asyncio
from wardbuddy.clinical_tutor import ClinicalTutor
from wardbuddy.utils import format_response
from wardbuddy.learning_context import setup_logger


logger = setup_logger(__name__)

ImportError: cannot import name 'ClinicalTutor' from 'wardbuddy.clinical_tutor' (C:\Users\deepa\OneDrive\Documents\StudyBuddy\wardbuddy\wardbuddy\clinical_tutor.py)

## Learning Interface

This module provides the user interface for the clinical learning system, including:
 * Case presentation and feedback
 * Learning preference configuration
 * Session management
 * Progress visualization

In [ ]:
class LearningInterface:
    """
    Gradio interface for clinical learning interactions.
    
    The interface provides:
    - Case presentation input and feedback
    - Learning preference configuration
    - Session history management
    - Progress tracking visualization
    
    Examples:
        >>> interface = LearningInterface()
        >>> demo = interface.create_interface()
        >>> demo.launch()
    """
    
    def __init__(
        self,
        context_path: Optional[Path] = None,
        theme: str = "default"
    ):
        """
        Initialize learning interface.
        
        Args:
            context_path: Optional path for context persistence
            theme: Gradio theme name
        """
        self.tutor = ClinicalTutor(context_path)
        self.theme = theme
        self.context_path = context_path
        logger.info("Learning interface initialized")
    
    async def process_input(
        self,
        message: str,
        history: List[Tuple[str, str]],
        feedback_focus: Dict[str, float]
    ) -> Tuple[List[Tuple[str, str]], str]:
        """
        Process user input and generate response.
        
        Args:
            message: User input message
            history: Conversation history
            feedback_focus: Current feedback preferences
            
        Returns:
            tuple: (updated_history, system_message)
        """
        try:
            if not message.strip():
                return history, "Please provide your case or question."
            
            # Update feedback preferences
            self.tutor.update_feedback_preferences(feedback_focus)
            
            # Process the interaction
            response = await self.tutor.process_interaction(message)
            
            # Format response for display
            formatted_response = format_response(response)
            
            history.append((message, formatted_response))
            return history, ""
            
        except Exception as e:
            logger.error(f"Error processing input: {str(e)}")
            return history, "An error occurred. Please try again."
    
    def update_preferences(
        self,
        clinical_reasoning: float,
        medical_knowledge: float,
        presentation_skills: float,
        differential_building: float
    ) -> Dict[str, float]:
        """
        Update feedback preferences.
        
        Args:
            clinical_reasoning: Focus weight for clinical reasoning
            medical_knowledge: Focus weight for medical knowledge
            presentation_skills: Focus weight for presentation skills
            differential_building: Focus weight for differential diagnosis
            
        Returns:
            dict: Updated preferences
        """
        preferences = {
            "clinical_reasoning": clinical_reasoning,
            "medical_knowledge": medical_knowledge,
            "presentation_skills": presentation_skills,
            "differential_building": differential_building
        }
        
        self.tutor.update_feedback_preferences(preferences)
        return preferences
    
    def create_interface(self) -> gr.Blocks:
        """
        Create and configure the Gradio interface.
        """
        with gr.Blocks(
            title="Clinical Learning Assistant",
            theme=self.theme,
            css="styles.css"
        ) as interface:
            
            # Header
            with gr.Row():
                gr.Markdown(
                    """# Clinical Learning Assistant
                    Enhance your clinical reasoning and presentation skills through 
                    adaptive feedback and guided practice."""
                )
            
            # Main content
            with gr.Tabs():
                # Case Discussion Tab
                with gr.Tab("Case Discussion"):
                    with gr.Row():
                        # Left column - Input and history
                        with gr.Column(scale=2):
                            chatbot = gr.Chatbot(
                                label="Learning Session",
                                height=500,
                                show_label=True
                            )
                            
                            with gr.Row():
                                msg = gr.Textbox(
                                    label="Present your case or ask a question",
                                    placeholder=(
                                        "Present your case as you would to your supervisor:\n"
                                        "- Start with the chief complaint\n"
                                        "- Include relevant history and findings\n"
                                        "- Share your assessment and plan"
                                    ),
                                    lines=5
                                )
                            
                            with gr.Row():
                                clear = gr.Button("Clear Session")
                                retry = gr.Button("Retry Last")
                        
                        # Right column - Current focus areas
                        with gr.Column(scale=1):
                            gr.Markdown("### Learning Focus Areas")
                            gr.Markdown(
                                "Adjust these sliders to emphasize different aspects "
                                "of feedback for your case presentations."
                            )
                            
                            feedback_focus = {
                                "clinical_reasoning": gr.Slider(
                                    minimum=0.5,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Clinical Reasoning",
                                    info="Emphasis on diagnostic process and medical decision-making"
                                ),
                                "medical_knowledge": gr.Slider(
                                    minimum=0.5,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Medical Knowledge",
                                    info="Focus on relevant medical concepts and evidence"
                                ),
                                "presentation_skills": gr.Slider(
                                    minimum=0.5,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Presentation Skills",
                                    info="Structure, clarity, and communication effectiveness"
                                ),
                                "differential_building": gr.Slider(
                                    minimum=0.5,
                                    maximum=2.0,
                                    value=1.0,
                                    step=0.1,
                                    label="Differential Building",
                                    info="Development of comprehensive differentials"
                                )
                            }
                
                # Learning Preferences Tab
                with gr.Tab("Learning Preferences"):
                    with gr.Row():
                        with gr.Column():
                            gr.Markdown("### Customize Your Learning Experience")
                            
                            with gr.Row():
                                learning_pace = gr.Radio(
                                    choices=["Gentle", "Moderate", "Challenging"],
                                    value="Moderate",
                                    label="Learning Pace",
                                    info="Adjust the depth and intensity of feedback"
                                )
                            
                            with gr.Row():
                                specialties = gr.CheckboxGroup(
                                    choices=[
                                        "Internal Medicine",
                                        "Emergency Medicine",
                                        "Surgery",
                                        "Pediatrics",
                                        "OB/GYN"
                                    ],
                                    label="Specialty Focus Areas",
                                    info="Select relevant specialties for your current rotation"
                                )
                            
                            update_prefs = gr.Button("Update Preferences")
                
                # Progress Tab
                with gr.Tab("Progress"):
                    with gr.Row():
                        gr.Markdown("### Your Learning Progress")
                        # Will add visualization components in future iterations
                        gr.Markdown(
                            "Progress tracking features coming soon:\n"
                            "- Case presentation history\n"
                            "- Skill development over time\n"
                            "- Identified learning opportunities\n"
                            "- Specialty-specific progress"
                        )
            
            # Event handlers
            msg.submit(
                self.process_input,
                inputs=[msg, chatbot] + list(feedback_focus.values()),
                outputs=[chatbot, msg]
            )
            
            clear.click(
                lambda: ([], ""),
                outputs=[chatbot, msg]
            )
            
            retry.click(
                lambda h: (h[:-1], h[-1][0]) if h else (h, ""),
                inputs=[chatbot],
                outputs=[chatbot, msg]
            )
            
            update_prefs.click(
                self.update_preferences,
                inputs=[
                    feedback_focus["clinical_reasoning"],
                    feedback_focus["medical_knowledge"],
                    feedback_focus["presentation_skills"],
                    feedback_focus["differential_building"]
                ],
                outputs=[]
            )
        
        return interface

## Launch Function

In [ ]:
#| export
async def launch_learning_interface(
    port: Optional[int] = None,
    context_path: Optional[Path] = None,
    share: bool = False,
    theme: str = "default"
) -> None:
    """
    Launch the learning interface application.
    
    Args:
        port: Optional port number
        context_path: Optional path for context persistence
        share: Whether to create a public link
        theme: Gradio theme name
    """
    try:
        interface = LearningInterface(context_path, theme)
        app = interface.create_interface()
        app.launch(
            server_port=port,
            share=share
        )
        logger.info(f"Interface launched on port: {port}")
    except Exception as e:
        logger.error(f"Error launching interface: {str(e)}")
        raise


## Custom CSS

In [ ]:
#| export
def write_css():
    """Create custom CSS file for the interface"""
    css = """
    .gr-button {
        border-radius: 8px;
        padding: 10px 20px;
    }
    
    .gr-button:hover {
        background-color: #2c5282;
        color: white;
    }
    
    .message {
        padding: 15px;
        border-radius: 10px;
        margin: 5px 0;
    }
    
    .user-message {
        background-color: #e2e8f0;
    }
    
    .assistant-message {
        background-color: #ebf8ff;
    }
    """
    
    with open("styles.css", "w") as f:
        f.write(css)


## Tests

In [ ]:
async def test_learning_interface():
    """Test learning interface functionality"""
    interface = LearningInterface()
    
    # Test input processing
    history = []
    feedback_focus = {
        "clinical_reasoning": 1.0,
        "medical_knowledge": 1.0,
        "presentation_skills": 1.0,
        "differential_building": 1.0
    }
    
    new_history, msg = await interface.process_input(
        "28yo M with chest pain",
        history,
        feedback_focus
    )
    
    assert len(new_history) == 1
    assert isinstance(new_history[0], tuple)
    assert len(new_history[0]) == 2
    
    # Test preference updates
    prefs = interface.update_preferences(1.5, 1.0, 1.0, 1.2)
    assert prefs["clinical_reasoning"] == 1.5
    assert prefs["differential_building"] == 1.2
    
    print("Interface tests passed!")

# Run tests
if __name__ == "__main__":
    write_css()
    asyncio.run(test_learning_interface())
